### Install enoslib (+ small setup)

In [ ]:
#!pip install -U pip
#!pip install enoslib
#!pip install python-grid5000

In [ ]:
# need for credentials in .env
#!echo -e "username: $(head -n 1 .env)\npassword: $(head -n 2 .env | tail -n 1)" > ~/.python-grid5000.yaml

In [41]:
import enoslib as en
from grid5000 import Grid5000
from pathlib import Path

conf_file = Path.home() / ".python-grid5000.yaml"
gk = Grid5000.from_yaml(conf_file)
gk.sites.list() # if credentials were written adequatly in .env, this should work

[<Site uid:grenoble>,
 <Site uid:lille>,
 <Site uid:luxembourg>,
 <Site uid:lyon>,
 <Site uid:nancy>,
 <Site uid:nantes>,
 <Site uid:rennes>,
 <Site uid:sophia>]

### Launch machines

In [43]:
job_name = 'dev'

conf = (
    en.G5kConf.from_settings(
        job_name=job_name,
        job_type=["deploy"],
        key="/home/adrien/.ssh/no_pass.pub",
        env_name="ubuntu2204-min",
        walltime="2:00:00"
    )    
    #.add_machine(roles=["client"], cluster="econome", nodes=1)
    .add_machine(roles=["server"], cluster="gros", nodes=1)
)

provider = en.G5k(conf)

roles, networks = provider.init()

Output()

Finished 1 tasks (Check deployment) on {'gros-90.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

Finished 1 tasks (Run dhcp on the nodes) on {'gros-90.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

In [44]:
provider.jobs

[<Job uid:4003598 site:nancy state:running user:agiot>]

### mount local files on grid5000 

In [45]:
import subprocess

site = conf.sites[0]

batcmd=f'ssh {site}.g5k "cat .ssh/id_rsa.pub"'
public = subprocess.check_output(batcmd, shell=True).decode()
res = en.run_command(f'echo "{public}" >> .ssh/authorized_keys', roles=roles)


print("CHECK")
res = en.run_command(f'cat .ssh/authorized_keys', roles=roles)
print([r.stdout for r in res])

Output()

Finished 1 tasks (echo "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDUHznpUX9GWGSfcmVImjhhbJHG4ooRU
qY1Apgcd5HC+6Rrgb/w2cK1dEFf4UNuOQI6CHRBf7hKGnuVdaR0t0XG1qlKcY4C996bsHzzoFCcpZNhLwaGytqeaq1kAj
fGINrN062rfKybwaA+Kpd6t5KHt2Qd85UkqJfI8TC7aMn/gG+uQnqP0lofYWoiSSK96opKyZVYqWFzpq3hZ4Rrub200pV
BiYL20jp+gSaLUAVQ6HYYqQ4ZEIy8oxfOP9KQ5IAVI4o86cQChIbXQlxbmsrG48nKMi1Tt0U0jNlE200/45i0jQaAzOL1
q/rHCe+ghjvbkw1QFykZ6l5q+QvZsC1P Generated passwordless ssh key to move between sites and 
connect nodes
" >> .ssh/authorized_keys) on {'gros-90.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

CHECK


Finished 1 tasks (cat .ssh/authorized_keys) on {'gros-90.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

['# Kadeploy key (v2013) see Bug 4923\nssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDFLLf3hDGtW56OMMw4uKg/5No2NfmsNMtCR212kiJubuGMC+J8yUFLZOicW0Y+hY5ESLcSVmmoJ5bdLyrtBZBB9EuS18ZIa3c4Lgnsb3HBlwsgLlBcl0kCWICqBs3zsAweIUW9V/vAfCSbReGR/vKqOwVdHr9Kmd3i46MKEZh/Do5jgNiHtF+d6IHLOgoq71KW9ulAdCV6VZic1S92bD766LQdjjgiSvsMBTxeN3eoFzEWtGmnD/q2UrIv4GOZjok3Q9nGCFK56oVssBapZmeO7BrJDh08WoCnDUMZUb7TezWG2q81t0rVciIosB8jjE1BOrtyK3d6UcUwFfkbA8clmG1HU9tSURVmwxoCKbxCYwwyJSmVMIV3aetzV1ifj8599EyxrihjMCFyodThIfrfJ93skktlYomI+qVlfpFjev0B9UBeJ5SVmKqZq2g6YlLYQwd2ri4kGA66qtD1ktdlfVJfxoEkiqjPMAGH5jp0oxj4c18b1rVqhy5lxScbIFbPdbsMhaB2leEfagumA/uUFgq+0HbGKFGYvmCI2J7CFxPvmemrMpjOj4GRpRqucb3gUGJj3tpaxUeTK5AIg1QFPvGHIK667pbAVsU0YW9HqG0Y7G09CNqY9eguftQ4wY6on0+7JpahahHKRupy0A3Hezv6YyGih7junXIIB5/vsQ== deploy@grid5000\nssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDByvHf+UfzmivK98UtCJLcR3wfCpmHGerr9gdGoadDvpZlX9gQC/9oB1nt+Xqp7n5BPnK6U0E8jlCg2QgjFGz8+xKuyot58gQaPBRdeNfD4xa4xGknEYSXfnuB0NUz/AFcHwEa965lIt0v2/r2kejMRew2+CTzyFYBu3Q/NvY4cQmWQ55xfP7

In [39]:
import os
for role in roles:
  hosts = [_.alias.split('.')[0] for _ in roles[role]]
  for host in hosts:
    os.system(f'bash mount_sshfs.sh {site} {host} {role}')

econome-5


### Setup the machines

In [11]:
with en.actions(roles=roles) as p:    
    p.shell("apt update")

Output()

Finished 1 tasks (shell) on {'gros-63.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

In [20]:
print([r.stdout for r in res])

['# Kadeploy key (v2013) see Bug 4923\nssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDFLLf3hDGtW56OMMw4uKg/5No2NfmsNMtCR212kiJubuGMC+J8yUFLZOicW0Y+hY5ESLcSVmmoJ5bdLyrtBZBB9EuS18ZIa3c4Lgnsb3HBlwsgLlBcl0kCWICqBs3zsAweIUW9V/vAfCSbReGR/vKqOwVdHr9Kmd3i46MKEZh/Do5jgNiHtF+d6IHLOgoq71KW9ulAdCV6VZic1S92bD766LQdjjgiSvsMBTxeN3eoFzEWtGmnD/q2UrIv4GOZjok3Q9nGCFK56oVssBapZmeO7BrJDh08WoCnDUMZUb7TezWG2q81t0rVciIosB8jjE1BOrtyK3d6UcUwFfkbA8clmG1HU9tSURVmwxoCKbxCYwwyJSmVMIV3aetzV1ifj8599EyxrihjMCFyodThIfrfJ93skktlYomI+qVlfpFjev0B9UBeJ5SVmKqZq2g6YlLYQwd2ri4kGA66qtD1ktdlfVJfxoEkiqjPMAGH5jp0oxj4c18b1rVqhy5lxScbIFbPdbsMhaB2leEfagumA/uUFgq+0HbGKFGYvmCI2J7CFxPvmemrMpjOj4GRpRqucb3gUGJj3tpaxUeTK5AIg1QFPvGHIK667pbAVsU0YW9HqG0Y7G09CNqY9eguftQ4wY6on0+7JpahahHKRupy0A3Hezv6YyGih7junXIIB5/vsQ== deploy@grid5000\nssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDByvHf+UfzmivK98UtCJLcR3wfCpmHGerr9gdGoadDvpZlX9gQC/9oB1nt+Xqp7n5BPnK6U0E8jlCg2QgjFGz8+xKuyot58gQaPBRdeNfD4xa4xGknEYSXfnuB0NUz/AFcHwEa965lIt0v2/r2kejMRew2+CTzyFYBu3Q/NvY4cQmWQ55xfP7

### Kill every last one of them

In [40]:
# Release all Grid'5000 resources
provider.destroy()